## Introduction
In this example, we are building a GlassFlow pipeline to encrypt email address from events data in real time. 

## Pre-requisite

- Create your free GlassFlow account via the [GlassFlow WebApp](https://app.glassflow.dev)
- Get your Personal Access Token to authorize the python sdk to interact with GlassFlow Cloud
- Run `pip install -r requirements.txt` to install the needed python dependencies for this example

In [ ]:
import glassflow

In [ ]:
# method to create an AES_KEY
def create_key():
    import secrets
    key = secrets.token_bytes(16)
    return key.hex()

In [ ]:
# Please edit this variable with your own personal access token from https://app.glassflow.dev/profile

In [ ]:
glassflow_personal_access_token = ""

In [ ]:
# Get Glassflow client and use the personal_access_token to authorize the client to connect with GlassFlow cloud

In [ ]:
client = glassflow.GlassFlowClient(personal_access_token=glassflow_personal_access_token)

In [ ]:
# use one of the existing glassflow space to put the pipeline.

In [ ]:
spaces = client.list_spaces()
space = spaces.spaces[0]

In [ ]:
print(space['name'])
print(space['id'])

In [ ]:
# read the transform.py file which has the python code that we submit to the pipeline

In [ ]:
with open("transform.py", 'r') as file:
    transform_content = file.read()
print(transform_content)

In [ ]:
# Create a AES_KEY and add it as an environment variable to the pipeline

In [ ]:
aes_key = create_key()

In [ ]:
env_vars = [{
      "name": "AES_KEY",
      "value": aes_key
    }]

In [ ]:
# The transform code depends on a python library. Add a requirements.txt to the pipeline

In [ ]:
with open("requirements.txt", 'r') as file:
    requirements_txt = file.read()
print(requirements_txt)

In [ ]:
# Create the pipeline with transform.py file and name it `email-encryption` in the main space
pipeline = client.create_pipeline(name="email-encryption-4", transformation_file='transform.py',
                                  space_id=space['id'], env_vars=env_vars, requirements=requirements_txt)

### Publish events to the pipeline

In [ ]:
# Create a random data generator to generate some dummy events data
from faker import Faker
def random_datagen():
    fake = Faker()
    return {
        "name": fake.name(),
        "email": fake.email(),
        "id": fake.uuid4()
    }

In [ ]:
data_source = pipeline.get_source()

In [ ]:
# Generate some data and send it to the pipeline. Store it locally to compare
input_data = []
for i in range(10):
    d = random_datagen()
    input_data.append(d)
    r = data_source.publish(d)

In [ ]:
display(input_data)

### Consume events from the pipeline 
Get the sink to consume transformed events from the pipeline

In [ ]:
data_sink = pipeline.get_sink()

In [ ]:
output_data = []
for i in range(10):
    resp = data_sink.consume()
    output_data.append(resp.json())

In [ ]:
display(output_data)

In [ ]:
## Explore the pipeline on the web-UI
pipeline_url = f"https://app.glassflow.dev/pipelines/{pipeline.id}"
print(pipeline_url)